## Подготовка тестовой выборки для проверки NER-моделей Natasha и DeepPavlov

```
для проверки NER-ов:
    1. берем из папки data/processed/test1_data default.tar.xz
    2. подгружаем его в datasets
    2. Создаем из него DataFrame, в котором в поле spans буду пары значений: [[сущность, тэг],]
```

In [1]:
from datasets import load_dataset, load_metric, ClassLabel, Sequence
from pathlib import Path
import random
import pandas as pd
from IPython.display import display, HTML
from deeppavlov.models.tokenizers.nltk_moses_tokenizer import NLTKMosesTokenizer, MosesDetokenizer 
from razdel import sentenize, tokenize

[nltk_data] Downloading package punkt to /home/akhmadiev/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/akhmadiev/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package perluniprops to
[nltk_data]     /home/akhmadiev/nltk_data...
[nltk_data]   Package perluniprops is already up-to-date!
[nltk_data] Downloading package nonbreaking_prefixes to
[nltk_data]     /home/akhmadiev/nltk_data...
[nltk_data]   Package nonbreaking_prefixes is already up-to-date!


In [2]:
task = "ner" # Should be one of "ner", "pos" or "chunk"
model_checkpoint = "bert-base-multilingual-uncased"#"mrm8488/distilbert-base-multi-cased-finetuned-typo-detection" "distilbert-base-uncased" 
batch_size = 16

In [4]:
BASE_DIR = Path().cwd().parent
wikiann_dl_dir = BASE_DIR.joinpath("data/interim")

In [10]:
wikiann_dl_dir

PosixPath('/media/akhmadiev/528807968807782D/Develop/PycharmProjects/ld_learn_ner/data/interim')

In [5]:
# подгружаекм dataset


import shutil
shutil.rmtree(str(wikiann_dl_dir.joinpath('cache')))
import os

os.mkdir(str(wikiann_dl_dir.joinpath('cache')))


datasets = load_dataset(str(wikiann_dl_dir.joinpath("wikiann_local.py")), data_files=str(wikiann_dl_dir.joinpath("default.tar.xz")), 
                        cache_dir=str(wikiann_dl_dir.joinpath('cache')))

Using custom data configuration default-8f6ad73a717029a6


Dataset wikiann downloaded and prepared to /media/akhmadiev/528807968807782D/Develop/PycharmProjects/ld_learn_ner/data/interim/cache/wikiann/default-8f6ad73a717029a6/1.1.0/17ccb6068a962cf083b84a313689d0878d2588d414a754548255cbc445bb1dc2. Subsequent calls will reuse this data.


In [6]:
datasets

DatasetDict({
    validation: Dataset({
        features: ['tokens', 'ner_tags', 'langs', 'spans'],
        num_rows: 20000
    })
    test: Dataset({
        features: ['tokens', 'ner_tags', 'langs', 'spans'],
        num_rows: 20000
    })
    train: Dataset({
        features: ['tokens', 'ner_tags', 'langs', 'spans'],
        num_rows: 20000
    })
})

In [7]:
# преобразуем dataset в DataFarme
dataset = datasets["test"]
df = pd.DataFrame(dataset)
for column, typ in dataset.features.items():
    if isinstance(typ, ClassLabel):
        df[column] = df[column].transform(lambda i: typ.names[i])
    elif isinstance(typ, Sequence) and isinstance(typ.feature, ClassLabel):
        df[column] = df[column].transform(lambda x: [typ.feature.names[i] for i in x])
display(HTML(df.head().to_html()))

,langs,ner_tags,spans,tokens
0,"[ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru]","[O, B-ORG, I-ORG, I-ORG, I-ORG, I-ORG, I-ORG, I-ORG, O, O, O, B-LOC, I-LOC, I-LOC, I-LOC, I-LOC, O, O, O, B-INNKPP, O, B-INNKPP, O, O, B-STAT, O, O, B-RSKS]","[ORG: МУП МП ГОРОДСКОГО ОКРУГА САМАРА КРАСНОГЛИНСКИЕ БАНИ, LOC: Брянская область , город Новозыбков, INNKPP: 100601303468, INNKPP: 632501001, STAT: +7-830-199-8031, RSKS: 30310411361955797442]","[Инвестор, МУП, МП, ГОРОДСКОГО, ОКРУГА, САМАРА, КРАСНОГЛИНСКИЕ, БАНИ, Адрес, филиала, банка, Брянская, область, ,, город, Новозыбков, Банк, Инвестор, ИНН/КПП, 100601303468, /, 632501001, тел, ., +7-830-199-8031, корр., счет, 30310411361955797442]"
1,"[ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru]","[O, B-ORG, I-ORG, I-ORG, I-ORG, I-ORG, I-ORG, I-ORG, I-ORG, I-ORG, I-ORG, I-ORG, O, O, O, B-INNKPP, O, B-INNKPP, O, O, B-STAT, I-STAT, I-STAT, I-STAT, O, O, B-RSKS, O, O, O, B-ORG, I-ORG, I-ORG, I-ORG, I-ORG, I-ORG, I-ORG, I-ORG, I-ORG, I-ORG, I-ORG, I-ORG, B-INNKPP, O, B-INNKPP, O, B-STAT, O, O, O, B-LOC, I-LOC]","[ORG: ОО НИЖЕГОРОДСКАЯ РЕГИОНАЛЬНАЯ ОБЩЕСТВЕННАЯ ОРГАНИЗАЦИЯ ПРАВОВОЙ ПОДДЕРЖКИ БЫСТРЫЙ ЛЕГИТИМНЫЙ АРБИТРАЖ СПОРОВ, INNKPP: 601500724, INNKPP: 611501001, STAT: +8-( 845 ) -147-5726, RSKS: 42160059899708393439, ORG: КОММЕРЧЕСКИЙ БАНК "" КОМПАНИЯ РОЗНИЧНОГО КРЕДИТОВАНИЯ "" ( ОТКРЫТОЕ АКЦИОНЕРНОЕ ОБЩЕСТВО ), INNKPP: 183116686028, INNKPP: 163201001, STAT: 040557259, LOC: Чувашская республика]","[Поставщик, ОО, НИЖЕГОРОДСКАЯ, РЕГИОНАЛЬНАЯ, ОБЩЕСТВЕННАЯ, ОРГАНИЗАЦИЯ, ПРАВОВОЙ, ПОДДЕРЖКИ, БЫСТРЫЙ, ЛЕГИТИМНЫЙ, АРБИТРАЖ, СПОРОВ, ИНН, КПП, Субподрядчик, 601500724, /, 611501001, тел, ., +8-(, 845, ), -147-5726, счет, №, 42160059899708393439, Филиал, банка, З-чик, КОММЕРЧЕСКИЙ, БАНК, "", КОМПАНИЯ, РОЗНИЧНОГО, КРЕДИТОВАНИЯ, "", (, ОТКРЫТОЕ, АКЦИОНЕРНОЕ, ОБЩЕСТВО, ), 183116686028, /, 163201001, БИК, 040557259, Адрес, банка, Подрядчик, Чувашская, республика]"
2,"[ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru]","[O, B-ORG, I-ORG, I-ORG, I-ORG, I-ORG, I-ORG, I-ORG, I-ORG, I-ORG, I-ORG, B-INNKPP, B-LOC, I-LOC, I-LOC, I-LOC, I-LOC, I-LOC, O, O, O, B-ORG, I-ORG, I-ORG, I-ORG, I-ORG, O, O, B-STAT, I-STAT, I-STAT, I-STAT, O, B-STAT]","[ORG: СЕВЕРНОЕ ЛИНЕЙНОЕ ПРОИЗВОДСТВЕННОЕ УПРАВЛЕНИЕ МАГИСТРАЛЬНЫХ ГАЗОПРОВОДОВ ( СЕВЕРНОЕ ЛПУМГ ), INNKPP: 631226435882, LOC: ГОРОД МОСКВА УЛИЦА УСАЧЁВА ДОМ 24, ORG: ООО Банк "" Саратов "", STAT: 8 344 741 9083, STAT: 044175088]","[Субподрядчик, СЕВЕРНОЕ, ЛИНЕЙНОЕ, ПРОИЗВОДСТВЕННОЕ, УПРАВЛЕНИЕ, МАГИСТРАЛЬНЫХ, ГАЗОПРОВОДОВ, (, СЕВЕРНОЕ, ЛПУМГ, ), 631226435882, ГОРОД, МОСКВА, УЛИЦА, УСАЧЁВА, ДОМ, 24, Филиал, банка, Плательщик, ООО, Банк, "", Саратов, "", тел, ., 8, 344, 741, 9083, БИК, 044175088]"
3,"[ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru]","[O, B-ORG, I-ORG, O, O, B-INNKPP, O, B-INNKPP, B-LOC, I-LOC, I-LOC, I-LOC, I-LOC, I-LOC, I-LOC, I-LOC, I-LOC, I-LOC, I-LOC, I-LOC, I-LOC, I-LOC, I-LOC, I-LOC, O, B-RSKS, O, O, O, B-ORG, I-ORG, I-ORG, I-ORG, I-ORG, I-ORG, I-ORG, I-ORG, I-ORG, O, B-RSKS]","[ORG: ООО ДЮКА, INNKPP: 564500394, INNKPP: 231201001, LOC: 624134 , Свердловская обл , г Новоуральск , ул Чурина , д 12/2 , оф 20, RSKS: 44468253734659494408, ORG: АКЦИОНЕРНОЕ ОБЩЕСТВО КОММЕРЧЕСКИЙ БАНК "" КУБАНСКИЙ ТОРГОВЫЙ БАНК "", RSKS: 30446422624228206241]","[Инвестор, ООО, ДЮКА, ИНН, КПП, 564500394, /, 231201001, 624134, ,, Свердловская, обл, ,, г, Новоуральск, ,, ул, Чурина, ,, д, 12/2, ,, оф, 20, рс, 44468253734659494408, Филиал, банка, П-щик, АКЦИОНЕРНОЕ, ОБЩЕСТВО, КОММЕРЧЕСКИЙ, БАНК, "", КУБАНСКИЙ, ТОРГОВЫЙ, БАНК, "", корр/сч, 30446422624228206241]"
4,"[ru, ru, 

In [27]:
chank = ['ООО', '"', 'МВ', 'КАСТОМ', '"', 'ИНН', 'З-чик', '7713782321', 'КПП', 'Банк', 'Подрядчик', 'АО', 
         'НОКССБАНК', 'тел.', '+8', '(', '842', ')', '14', '08', '762', 'тел.', '+8-956-009-5122']

In [8]:
# на основе поля массива токенов создаем строку исходного текста
def make_str_from_tokens(chank):
    detokenizer = NLTKMosesTokenizer('ru').detokenizer
    tokens = detokenizer.detokenize([''.join(c)  for c in chank])
    return tokens

In [9]:
# восстанавливаем текст на основе поля tokens
df['text'] = df['tokens'].apply(make_str_from_tokens)

In [14]:
display(HTML(df.head().to_html()))

,langs,ner_tags,spans,tokens,text,teg_value
0,"[ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru]","[O, B-ORG, I-ORG, I-ORG, I-ORG, I-ORG, I-ORG, I-ORG, O, O, O, B-LOC, I-LOC, I-LOC, I-LOC, I-LOC, O, O, O, B-INNKPP, O, B-INNKPP, O, O, B-STAT, O, O, B-RSKS]","[ORG: МУП МП ГОРОДСКОГО ОКРУГА САМАРА КРАСНОГЛИНСКИЕ БАНИ, LOC: Брянская область , город Новозыбков, INNKPP: 100601303468, INNKPP: 632501001, STAT: +7-830-199-8031, RSKS: 30310411361955797442]","[Инвестор, МУП, МП, ГОРОДСКОГО, ОКРУГА, САМАРА, КРАСНОГЛИНСКИЕ, БАНИ, Адрес, филиала, банка, Брянская, область, ,, город, Новозыбков, Банк, Инвестор, ИНН/КПП, 100601303468, /, 632501001, тел, ., +7-830-199-8031, корр., счет, 30310411361955797442]","Инвестор МУП МП ГОРОДСКОГО ОКРУГА САМАРА КРАСНОГЛИНСКИЕ БАНИ Адрес филиала банка Брянская область, город Новозыбков Банк Инвестор ИНН/КПП 100601303468 / 632501001 тел. +7-830-199-8031 корр. счет 30310411361955797442","[[ORG, МУП МП ГОРОДСКОГО ОКРУГА САМАРА КРАСНОГЛИНСКИЕ БАНИ], [LOC, Брянская область , город Новозыбков], [INNKPP, 100601303468], [INNKPP, 632501001], [STAT, +7-830-199-8031], [RSKS, 30310411361955797442]]"
1,"[ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru]","[O, B-ORG, I-ORG, I-ORG, I-ORG, I-ORG, I-ORG, I-ORG, I-ORG, I-ORG, I-ORG, I-ORG, O, O, O, B-INNKPP, O, B-INNKPP, O, O, B-STAT, I-STAT, I-STAT, I-STAT, O, O, B-RSKS, O, O, O, B-ORG, I-ORG, I-ORG, I-ORG, I-ORG, I-ORG, I-ORG, I-ORG, I-ORG, I-ORG, I-ORG, I-ORG, B-INNKPP, O, B-INNKPP, O, B-STAT, O, O, O, B-LOC, I-LOC]","[ORG: ОО НИЖЕГОРОДСКАЯ РЕГИОНАЛЬНАЯ ОБЩЕСТВЕННАЯ ОРГАНИЗАЦИЯ ПРАВОВОЙ ПОДДЕРЖКИ БЫСТРЫЙ ЛЕГИТИМНЫЙ АРБИТРАЖ СПОРОВ, INNKPP: 601500724, INNKPP: 611501001, STAT: +8-( 845 ) -147-5726, RSKS: 42160059899708393439, ORG: КОММЕРЧЕСКИЙ БАНК "" КОМПАНИЯ РОЗНИЧНОГО КРЕДИТОВАНИЯ "" ( ОТКРЫТОЕ АКЦИОНЕРНОЕ ОБЩЕСТВО ), INNKPP: 183116686028, INNKPP: 163201001, STAT: 040557259, LOC: Чувашская республика]","[Поставщик, ОО, НИЖЕГОРОДСКАЯ, РЕГИОНАЛЬНАЯ, ОБЩЕСТВЕННАЯ, ОРГАНИЗАЦИЯ, ПРАВОВОЙ, ПОДДЕРЖКИ, БЫСТРЫЙ, ЛЕГИТИМНЫЙ, АРБИТРАЖ, СПОРОВ, ИНН, КПП, Субподрядчик, 601500724, /, 611501001, тел, ., +8-(, 845, ), -147-5726, счет, №, 42160059899708393439, Филиал, банка, З-чик, КОММЕРЧЕСКИЙ, БАНК, "", КОМПАНИЯ, РОЗНИЧНОГО, КРЕДИТОВАНИЯ, "", (, ОТКРЫТОЕ, АКЦИОНЕРНОЕ, ОБЩЕСТВО, ), 183116686028, /, 163201001, БИК, 040557259, Адрес, банка, Подрядчик, Чувашская, республика]","Поставщик ОО НИЖЕГОРОДСКАЯ РЕГИОНАЛЬНАЯ ОБЩЕСТВЕННАЯ ОРГАНИЗАЦИЯ ПРАВОВОЙ ПОДДЕРЖКИ БЫСТРЫЙ ЛЕГИТИМНЫЙ АРБИТРАЖ СПОРОВ ИНН КПП Субподрядчик 601500724 / 611501001 тел. +8-( 845) -147-5726 счет № 42160059899708393439 Филиал банка З-чик КОММЕРЧЕСКИЙ БАНК ""КОМПАНИЯ РОЗНИЧНОГО КРЕДИТОВАНИЯ"" (ОТКРЫТОЕ АКЦИОНЕРНОЕ ОБЩЕСТВО) 183116686028 / 163201001 БИК 040557259 Адрес банка Подрядчик Чувашская республика","[[ORG, ОО НИЖЕГОРОДСКАЯ РЕГИОНАЛЬНАЯ ОБЩЕСТВЕННАЯ ОРГАНИЗАЦИЯ ПРАВОВОЙ ПОДДЕРЖКИ БЫСТРЫЙ ЛЕГИТИМНЫЙ АРБИТРАЖ СПОРОВ], [INNKPP, 601500724], [INNKPP, 611501001], [STAT, +8-( 845 ) -147-5726], [RSKS, 42160059899708393439], [ORG, КОММЕРЧЕСКИЙ БАНК "" КОМПАНИЯ РОЗНИЧНОГО КРЕДИТОВАНИЯ "" ( ОТКРЫТОЕ АКЦИОНЕРНОЕ ОБЩЕСТВО )], [INNKPP, 183116686028], [INNKPP, 163201001], [STAT, 040557259], [LOC, Чувашская республика]]"
2,"[ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru]","[O, B-ORG, I-ORG, I-ORG, I-ORG, I-ORG, I-ORG, I-ORG, I-ORG, I-ORG, I-ORG, B-INNKPP, B-LOC, I-LOC, I-LOC, I-LOC, I-LOC, I-LOC, O, O, O, B-ORG, I-ORG, I-ORG, I-ORG, I-ORG, O, O, B-STAT, I-STAT, I-STAT, I-STAT, O, B-STAT]","[ORG: СЕВЕРНОЕ ЛИНЕЙНОЕ ПРОИЗВОДСТВЕННОЕ УПРАВЛЕНИЕ МАГИСТРАЛЬНЫХ ГАЗОПРОВОДОВ ( СЕВЕРНОЕ ЛПУМГ ), INNKPP: 631226435882, LOC: ГОРОД МОСКВА УЛИЦА УСАЧЁВА ДОМ 24, ORG: ООО Банк "" Саратов "", STAT: 8 344 741 9083, STAT: 044175088]","[Субподряд

In [12]:
# создаем формат [[tag, value]] для оценки качества других NER-моделей
def make_tag_value_from_spans(spans):
    i = 0
    result = []
#     if spans[0] == 'None':
#         return result
    while i < len(spans):
        s = spans[i].split(': ')
        if s[0] == 'ORG' and s[1].split()[0].upper() in ['ИП', 'ИНДИВИДУАЛЬНЫЙ']:
            s1 = spans[i+1].split(': ')
            if s1[0] == 'PER':
                result.append([s[0], "{} {}".format(s[1], s1[1])])
                i+=1
            else:
                result.append([s[0], s[1]])
        else:
            result.append([s[0], s[1]])
        i+=1
    return result

In [13]:
# восстанавливаем тэги и их значения
df['teg_value'] = df['spans'].apply(make_tag_value_from_spans)

In [15]:
df[['tokens', 'text', 'teg_value']].to_excel(BASE_DIR.joinpath('data/processed/test_model_example.xlsx'), index=False, engine='openpyxl')